In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Obesity"
cohort = "GSE158850"

# Input paths
in_trait_dir = "../../input/GEO/Obesity"
in_cohort_dir = "../../input/GEO/Obesity/GSE158850"

# Output paths
out_data_file = "../../output/preprocess/Obesity/GSE158850.csv"
out_gene_data_file = "../../output/preprocess/Obesity/gene_data/GSE158850.csv"
out_clinical_data_file = "../../output/preprocess/Obesity/clinical_data/GSE158850.csv"
json_path = "../../output/preprocess/Obesity/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch2']

# Extract background information and clinical data
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("\nSample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Skeletal muscle adaptations to an 8-week weight loss intervention in younger and older obese men"
!Series_summary	"Vastus lateralis Skeletal muscle transcriptomes were analyzed between 13 young  and 12 elderly obese men during a lifestyle intervention."
!Series_summary	"Lifestyle intervention: Individuals underwent 8-weeks of calorie-restriction of 20% below their daily energy requirement aerobic combined to two sessions of resistance exercise per weeks."
!Series_overall_design	"Two groups,young and elderly; two conditions for each individual, baseline (before lifestyle intervention) and after  lifestyle intervention."

Sample Characteristics Dictionary:
{0: ['tissue: Skeletal muscle'], 1: ['Sex: Male'], 2: ['age group: Young', 'age group: Elderly'], 3: ['disease state: obese'], 4: ['time point: baseline', 'time point: after lifestyle intervention']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains skeletal muscle transcriptomes
# which indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Obesity):
# From the sample characteristics, we have "disease state: obese" at key 3
# and "time point: baseline" vs "time point: after lifestyle intervention" at key 4
# The trait we're studying is Obesity, and we can infer it from the time point
# since we're comparing before and after a weight loss intervention
trait_row = 4  # Using time point as a proxy for obesity status

# For age:
# From the sample characteristics, we have "age group: Young" vs "age group: Elderly" at key 2
age_row = 2

# For gender:
# From the sample characteristics, we have "Sex: Male" at key 1, which indicates
# all participants are male (constant feature), so we consider it not available
gender_row = None

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert time point value to binary obesity status.
    
    Before intervention (baseline) = 1 (obese)
    After intervention = 0 (less obese)
    """
    if isinstance(value, str):
        value = value.lower().strip()
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        
        if "baseline" in value:
            return 1  # Obese at baseline
        elif "after" in value:
            return 0  # Less obese after intervention
    return None

def convert_age(value):
    """Convert age group value to binary age category.
    
    Young = 0
    Elderly = 1
    """
    if isinstance(value, str):
        value = value.lower().strip()
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        
        if "young" in value:
            return 0
        elif "elderly" in value:
            return 1
    return None

def convert_gender(value):
    """Convert gender value to binary.
    
    Female = 0
    Male = 1
    
    Not used in this dataset as all participants are male.
    """
    if isinstance(value, str):
        value = value.lower().strip()
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        
        if "female" in value:
            return 0
        elif "male" in value:
            return 1
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering and save information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data to a CSV file
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM4812195': [1.0, 0.0], 'GSM4812196': [0.0, 0.0], 'GSM4812197': [1.0, 1.0], 'GSM4812198': [0.0, 1.0], 'GSM4812199': [1.0, 0.0], 'GSM4812200': [0.0, 0.0], 'GSM4812201': [1.0, 1.0], 'GSM4812202': [0.0, 1.0], 'GSM4812203': [1.0, 0.0], 'GSM4812204': [0.0, 0.0], 'GSM4812205': [1.0, 1.0], 'GSM4812206': [0.0, 1.0], 'GSM4812207': [1.0, 0.0], 'GSM4812208': [0.0, 0.0], 'GSM4812209': [1.0, 1.0], 'GSM4812210': [0.0, 1.0], 'GSM4812211': [1.0, 0.0], 'GSM4812212': [0.0, 0.0], 'GSM4812213': [1.0, 1.0], 'GSM4812214': [0.0, 1.0], 'GSM4812215': [1.0, 0.0], 'GSM4812216': [0.0, 0.0], 'GSM4812217': [1.0, 1.0], 'GSM4812218': [0.0, 1.0], 'GSM4812219': [1.0, 0.0], 'GSM4812220': [0.0, 0.0], 'GSM4812221': [1.0, 1.0], 'GSM4812222': [0.0, 1.0], 'GSM4812223': [1.0, 0.0], 'GSM4812224': [0.0, 0.0], 'GSM4812225': [1.0, 1.0], 'GSM4812226': [0.0, 1.0], 'GSM4812227': [1.0, 0.0], 'GSM4812228': [0.0, 0.0], 'GSM4812229': [1.0, 1.0], 'GSM4812230': [0.0, 1.0], 'GSM4812231': [1.0, 0.0]

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['A_23_P100001', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074',
       'A_23_P100127', 'A_23_P100141', 'A_23_P100189', 'A_23_P100196',
       'A_23_P100203', 'A_23_P100220', 'A_23_P100240', 'A_23_P10025',
       'A_23_P100292', 'A_23_P100315', 'A_23_P100326', 'A_23_P100344',
       'A_23_P100355', 'A_23_P100386', 'A_23_P100392', 'A_23_P100420'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers in this dataset (like 'A_23_P100001') are Agilent probe IDs, not human gene symbols.
# These are probe identifiers from an Agilent microarray platform and need to be mapped to official gene symbols.
# These IDs follow the Agilent naming convention with format "A_23_P######" which are platform-specific probe IDs.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'SPOT_ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan], 'GO_ID': [nan, nan, nan, nan, nan], 'SEQUENCE': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# Step 1: Identify which columns in the gene annotation to use for mapping
# Looking at the gene_annotation preview, we need to match 'ID' from annotation with row indices in gene_data.
# The 'GENE_SYMBOL' column contains the gene symbols we want to map to.

# Step 2: Get a gene mapping dataframe from the gene annotation dataframe
mapping_data = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

# Step 3: Apply the gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, mapping_data)

# Normalize gene symbols to handle synonyms and aggregate rows with the same normalized symbol
gene_data = normalize_gene_symbols_in_index(gene_data)

# Save the processed gene expression data
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data has been processed and saved to {out_gene_data_file}")
print(f"Number of genes in the processed data: {len(gene_data)}")
print(f"Preview of processed gene data:")
print(gene_data.head(3))


Gene expression data has been processed and saved to ../../output/preprocess/Obesity/gene_data/GSE158850.csv
Number of genes in the processed data: 19447
Preview of processed gene data:
          GSM4812195  GSM4812196  GSM4812197  GSM4812198  GSM4812199  \
Gene                                                                   
A1BG       -6.346192   -6.284652   -6.555014   -5.325181   -5.702537   
A1BG-AS1   -0.094500   -0.286431    0.034552    0.555810   -0.188718   
A1CF       -2.590942   -2.309197   -2.679292   -2.385138   -2.853777   

          GSM4812200  GSM4812201  GSM4812202  GSM4812203  GSM4812204  ...  \
Gene                                                                  ...   
A1BG       -6.670208   -6.870791   -7.228066   -3.628051   -3.890610  ...   
A1BG-AS1    0.401974   -0.358381    0.251461    0.700686    0.158606  ...   
A1CF       -2.727571   -3.005896   -2.766465   -1.226089   -1.441245  ...   

          GSM4812233  GSM4812234  GSM4812235  GSM4812236  GSM481223

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data
# Note: normalized_gene_data was already created in Step 6 when we applied normalize_gene_symbols_in_index
# and saved to out_gene_data_file, so we can just use gene_data which is already normalized

# 2. Load the previously saved clinical data
clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
print(f"Loaded clinical data with shape: {clinical_df.shape}")

# 3. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values in the linked data systematically
linked_data = handle_missing_values(linked_data, trait)
print(f"After handling missing values, linked data shape: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are severely biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Is trait biased: {is_biased}")

# 6. Conduct final quality validation and save cohort information
note = "This dataset compares skeletal muscle adaptations before and after a weight loss intervention in young and elderly obese men."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 7. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies.")

Loaded clinical data with shape: (2, 48)
Linked data shape: (48, 19449)


After handling missing values, linked data shape: (48, 19449)
For the feature 'Obesity', the least common label is '1.0' with 24 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Obesity' in this dataset is fine.

Quartiles for 'Age':
  25%: 0.0
  50% (Median): 0.5
  75%: 1.0
Min: 0.0
Max: 1.0
The distribution of the feature 'Age' in this dataset is fine.

Is trait biased: False


Linked data saved to ../../output/preprocess/Obesity/GSE158850.csv
